In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_6/models/B5R2b5_5FP_1FC_fold6_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image4/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold4.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold4.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold6.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(658, 25)
(658, 25)
Normal:  (429, 25)
Abnormal:  (229, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,133,133,13,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,103.0,132.0,675.0,590.0,0.108818,0.146881,0.707317,0.657370,AB01 P4-2 C013.JPG,AB01 P4-2 C013.JPG
1,1,1,134,134,13,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,283.0,142.0,384.0,399.0,0.294227,0.158833,0.398510,0.444223,AB01 P2 C013.JPG,AB01 P2 C013.JPG
2,2,2,135,135,13,P5,P51,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,434.0,530.0,0.248619,0.113017,0.445672,0.589641,AB01 P5-1 C013.JPG,AB01 P5-1 C013.JPG
3,3,3,136,136,13,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,274.0,159.0,432.0,415.0,0.287594,0.176762,0.453008,0.462151,AB01 P1 C013.JPG,AB01 P1 C013.JPG
4,4,4,137,137,13,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,256.0,164.0,546.0,436.0,0.277132,0.182738,0.591085,0.486056,AB01 P4-1 C013.JPG,AB01 P4-1 C013.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Views',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 658 validated image filenames belonging to 5 classes.
{0: 'FP-A', 1: 'FP-B', 2: 'FP-', 3: 'FP-D', 4: 'FP-E'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[ 8.81114900e-02,  8.49786922e-02, -1.59340903e-01, ...,
        -7.92905241e-02,  2.53597170e-01,  3.68812561e-01],
       [-3.60425822e-02,  1.76503554e-01,  4.06562716e-01, ...,
        -1.24892198e-01, -1.51273757e-01,  1.60442963e-01],
       [-2.01365933e-01, -1.25678346e-01, -8.16177651e-02, ...,
         2.06031263e-01, -4.91392910e-02, -1.62657425e-01],
       ...,
       [-1.73923016e-01,  4.40429896e-02,  8.73449221e-02, ...,
         6.39044285e-01,  3.19323599e-01, -1.58037052e-01],
       [-2.14643821e-01,  7.55352154e-02, -6.69775764e-03, ...,
         4.82658833e-01,  4.13239330e-01, -1.57191411e-01],
       [-7.84987956e-02,  3.14536970e-04, -2.16393963e-01, ...,
        -1.20677114e-01, -1.97189599e-01,  8.31182152e-02]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.088111,0.084979,-0.159341,-0.121146,0.108318,0.055693,-0.206385,0.670532,0.161083,0.169354,...,0.350940,-0.129352,0.228695,0.431978,-0.006836,0.428255,0.634549,-0.079291,0.253597,0.368813
1,-0.036043,0.176504,0.406563,-0.057735,0.059977,0.076648,0.514208,0.203904,-0.197568,-0.044129,...,-0.120179,-0.171064,-0.233643,-0.119508,-0.092788,0.047489,0.043831,-0.124892,-0.151274,0.160443
2,-0.201366,-0.125678,-0.081618,0.060315,-0.203556,-0.190703,-0.189451,-0.046204,0.128575,-0.220287,...,0.212207,0.052209,0.303637,-0.168448,-0.211645,-0.222409,-0.213920,0.206031,-0.049139,-0.162657
3,0.229069,0.040427,-0.096845,0.212402,0.290006,-0.223279,0.664260,0.584298,-0.120318,0.126017,...,-0.179770,-0.217699,-0.233050,-0.184653,-0.117410,0.065853,0.108393,-0.093355,-0.220048,0.243307
4,0.098194,-0.125541,-0.193168,-0.103063,0.069133,-0.095026,-0.110996,0.384701,-0.191665,0.492039,...,0.002147,-0.206956,-0.013455,0.588839,-0.142851,0.520026,0.513894,0.154666,0.113198,0.568250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,0.135335,-0.133799,-0.121256,-0.070816,0.102646,-0.168273,-0.107410,0.416860,-0.037102,0.144250,...,0.181174,-0.078026,0.371846,0.538801,-0.189302,0.464991,0.726382,-0.010262,0.267716,0.228705
654,-0.008360,-0.063737,-0.107836,0.031093,0.314323,-0.106467,-0.205384,0.576752,-0.048196,0.103558,...,0.015915,-0.184080,0.081992,0.240374,0.005061,0.131541,0.319574,-0.147122,0.105844,0.376450
655,-0.173923,0.044043,0.087345,0.198223,-0.221117,-0.159845,-0.013978,-0.045500,0.718877,-0.212385,...,0.342157,0.525208,0.296091,-0.024287,0.099829,-0.169990,-0.175388,0.639044,0.319324,-0.158037
656,-0.214644,0.075535,-0.006698,-0.053203,-0.189083,-0.182044,-0.178347,0.151136,0.400567,-0.226095,...,0.208605,-0.138147,0.345609,-0.066893,-0.182873,-0.118322,-0.210180,0.482659,0.413239,-0.157191


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,0.088111,0.084979,-0.159341,-0.121146,0.108318,0.055693,-0.206385,0.670532,0.161083,0.169354,...,-0.006836,0.428255,0.634549,-0.079291,0.253597,0.368813,Abnormal,P42,FP-B,AB01
1,-0.036043,0.176504,0.406563,-0.057735,0.059977,0.076648,0.514208,0.203904,-0.197568,-0.044129,...,-0.092788,0.047489,0.043831,-0.124892,-0.151274,0.160443,Abnormal,P2,FP-A,AB01
2,-0.201366,-0.125678,-0.081618,0.060315,-0.203556,-0.190703,-0.189451,-0.046204,0.128575,-0.220287,...,-0.211645,-0.222409,-0.213920,0.206031,-0.049139,-0.162657,Abnormal,P51,FP-C,AB01
3,0.229069,0.040427,-0.096845,0.212402,0.290006,-0.223279,0.664260,0.584298,-0.120318,0.126017,...,-0.117410,0.065853,0.108393,-0.093355,-0.220048,0.243307,Abnormal,P1,FP-A,AB01
4,0.098194,-0.125541,-0.193168,-0.103063,0.069133,-0.095026,-0.110996,0.384701,-0.191665,0.492039,...,-0.142851,0.520026,0.513894,0.154666,0.113198,0.568250,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,0.135335,-0.133799,-0.121256,-0.070816,0.102646,-0.168273,-0.107410,0.416860,-0.037102,0.144250,...,-0.189302,0.464991,0.726382,-0.010262,0.267716,0.228705,Normal,P32,FP-B,Normal
654,-0.008360,-0.063737,-0.107836,0.031093,0.314323,-0.106467,-0.205384,0.576752,-0.048196,0.103558,...,0.005061,0.131541,0.319574,-0.147122,0.105844,0.376450,Normal,P42,FP-B,Normal
655,-0.173923,0.044043,0.087345,0.198223,-0.221117,-0.159845,-0.013978,-0.045500,0.718877,-0.212385,...,0.099829,-0.169990,-0.175388,0.639044,0.319324,-0.158037,Normal,P52,FP-C,Normal
656,-0.214644,0.075535,-0.006698,-0.053203,-0.189083,-0.182044,-0.178347,0.151136,0.400567,-0.226095,...,-0.182873,-0.118322,-0.210180,0.482659,0.413239,-0.157191,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_5FP_RF_5FP_fold6_1.csv')